In [1]:
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalizationV2

In [2]:
train_path = '../data/black_box_combined/seg_train/'
test_path = '../../normal/archive/seg_test/'

In [3]:
train_datagen = ImageDataGenerator(
                    rescale = 1/255,
                    shear_range = 0.2,
                    zoom_range = 0.2,
                    horizontal_flip = True,
                    vertical_flip = False)
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (64, 64),
                                                 class_mode = 'categorical')

Found 28068 images belonging to 6 classes.


In [4]:
test_datagen = ImageDataGenerator(rescale = 1/255)
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (64, 64),
                                            class_mode = 'categorical')

Found 3000 images belonging to 6 classes.


In [5]:
cnn = Sequential()
cnn

Metal device set to: Apple M1


2022-08-29 22:50:49.895277: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-29 22:50:49.895429: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
cnn.add(Convolution2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(MaxPooling2D(pool_size=(2,2)))

cnn.add(Convolution2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(MaxPooling2D(pool_size=(2,2)))

cnn.add(Convolution2D(filters=128, kernel_size=3, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(MaxPooling2D(pool_size=(2,2)))

cnn.add(Convolution2D(filters=256, kernel_size=3, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(MaxPooling2D(pool_size=(2,2)))

cnn.add(Dropout(0.4)),

cnn.add(Flatten())

cnn.add(Dense(units=256, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(Dropout(0.25))

cnn.add(Dense(units=128, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(Dropout(0.25))

cnn.add(Dense(units=64, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(Dropout(0.25))

cnn.add(Dense(units=24, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(Dropout(0.25))

cnn.add(Dense(units=6, activation='softmax'))

In [7]:
cnn.compile(optimizer = 'sgd', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [8]:
%%time
cnn.fit(training_set, steps_per_epoch= len(training_set), validation_data = test_set, validation_steps=len(test_set), epochs = 10)

Epoch 1/10


2022-08-29 22:50:50.510181: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-29 22:50:51.534544: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


878/878 [==============================] - ETA: 0s - loss: 1.3669 - accuracy: 0.4655

2022-08-29 22:51:58.637590: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


878/878 [==============================] - 71s 79ms/step - loss: 1.3669 - accuracy: 0.4655 - val_loss: 1.1368 - val_accuracy: 0.5443
Epoch 2/10
878/878 [==============================] - 70s 80ms/step - loss: 1.0378 - accuracy: 0.5975 - val_loss: 0.8208 - val_accuracy: 0.6833
Epoch 3/10
878/878 [==============================] - 81s 93ms/step - loss: 1.0372 - accuracy: 0.6041 - val_loss: 0.9136 - val_accuracy: 0.6450
Epoch 4/10
878/878 [==============================] - 87s 98ms/step - loss: 0.9223 - accuracy: 0.6542 - val_loss: 0.9891 - val_accuracy: 0.6040
Epoch 5/10
878/878 [==============================] - 72s 81ms/step - loss: 0.8756 - accuracy: 0.6790 - val_loss: 0.9304 - val_accuracy: 0.6593
Epoch 6/10
878/878 [==============================] - 74s 85ms/step - loss: 0.7822 - accuracy: 0.7195 - val_loss: 0.8937 - val_accuracy: 0.6713
Epoch 7/10
878/878 [==============================] - 80s 91ms/step - loss: 0.9725 - accuracy: 0.6380 - val_loss: 1.0725 - val_accuracy: 0.6463
Epo

In [9]:
accuracy=cnn.evaluate(test_set, steps=len(test_set),verbose=2)
print('Accuracy of the model on the test set: ',accuracy[1])

94/94 - 2s - loss: 1.1187 - accuracy: 0.6310 - 2s/epoch - 25ms/step
Accuracy of the model on the test set:  0.6309999823570251
